In [38]:
import requests 
import os
import tiktoken

import openai
from openai import OpenAI

from dotenv import load_dotenv
load_dotenv(override=True)
OPENAI_API_KEY= os.getenv("OPENAI_KEY")

from tqdm.auto import tqdm 

In [39]:

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)


In [40]:
def search(query):
    boost = {'question': 3.0, 'section':0.5}

    results = index.search(
        query=query,
        #filter_dict={'course':'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    #print("total", len(results))
    #resulls
    return results

def build_prompt(query, search_results):

    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    QUESTION: {question}
    CONTEXT: 
    {context}
    """.strip()
    

    context = ""
    #get the context from 
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()    
    return prompt

def llm(prompt):

    try:
        completion = client.chat.completions.create(
                model="gpt-4o",
                messages=[{"role":"user","content":prompt}]
            )    
    except Error as e:
            if e.status == 429:  # HTTP status code for Rate Limit Exceeded
                print("Rate limit exceeded. Please wait and retry.")
            else:
                print(f"Error: {e}")
    return completion.choices[0].message.content

def openai(query,engine='gpt-3.5-turbo'):
    try:
        client = OpenAI(api_key=OPENAI_API_KEY)

        completion = client.chat.completions.create(
            model=engine,
            messages=[{"role":"user","content":query}]
        )    
    except Error as e:
        if e.status == 429:  # HTTP status code for Rate Limit Exceeded
            print("Rate limit exceeded. Please wait and retry.")
        else:
            print(f"Error: {e}")

    result = completion.choices[0].message.content

    return result

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query,search_results)
    answer = llm(prompt)
    return answer
     

In [41]:
from elasticsearch import Elasticsearch
ES_URL = 'http://localhost:9200'
es_client = Elasticsearch(ES_URL)
answer = es_client.info()

#es data
# for item in answer:
#     print (item,'-',answer[item])

#get the version-buildhash
print ('Q1- What''s the version.build_hash value?')
print(answer['version']['build_hash'])

Q1- Whats the version.build_hash value?
42f05b9372a9a4a470db3b52817899b99a76ee73


In [42]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course_questions"

# Function to check if an index exists
def index_exists(index_name):
    try:
        return es_client.indices.exists(index=index_name)
    except NotFoundError:
        return False

# Function to delete an index
def delete_index(index_name):
    try:
        es_client.indices.delete(index=index_name)
        print(f"Index '{index_name}' deleted successfully.")
    except NotFoundError:
        print(f"Index '{index_name}' not found.")

# Function to create a new index
def create_index(index_name, idx_settings):
    settings = idx_settings
    try:
        es_client.indices.create(index=index_name, body=settings)
        print(f"Index '{index_name}' created successfully.")
    except Exception as e:
        print(f"Failed to create index '{index_name}': {str(e)}")


if index_exists(index_name):
    # Index exists, delete it
    delete_index(index_name)

# Create the new index
create_index(index_name,index_settings)



Index 'course_questions' deleted successfully.
Index 'course_questions' created successfully.


In [43]:
#indexa arquivos
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:19<00:00, 47.65it/s]


In [44]:
print ('Q2- Which function do you use for adding your data to elastic?')
print('index')

Q2- Which function do you use for adding your data to elastic?
index


In [45]:
query = "How do I execute a command in a running docker container?"

def elastic_search(query,filter,size=5):

    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                "term": {
                    "course": filter
                }
            }
            }
        }
    }

    search_results = es_client.search(index=index_name, body=search_query)
    result_docs=[]

    for hit in search_results['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

#elastic_search(query,'fitro1')
    

In [46]:
query = "How do I execute a command in a running docker container?"
search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                # "filter": {
                # "term": {
                #     "course": "data-engineering-zoomcamp"
                # }   
                # }
        }
    }
}

search_results = es_client.search(index=index_name, body=search_query)

print ('Q3- What''s the score for the top ranking result?')
print(search_results['hits']['hits'][0]['_score'])

Q3- Whats the score for the top ranking result?
84.050095


In [47]:
search_results=elastic_search(query, 'machine-learning-zoomcamp',3)
#print (search_results)
third = search_results[2]['question']

print ('Q4- Now let''s only limit the questions to machine-learning-zoomcamp adding a filter to the qeury.\nReturn 3 results.\nWhat''s the 3rd question returned by the search engine?')
print(third)

Q4- Now lets only limit the questions to machine-learning-zoomcamp adding a filter to the qeury.
Return 3 results.
Whats the 3rd question returned by the search engine?
How do I copy files from a different folder into docker container’s working directory?


In [55]:
prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    QUESTION: {question}
    CONTEXT: 
    {context}
    """.strip()
    

context_template="""
Q:{question}
A:{text}
""".strip()

context_pieces=[]
for doc in search_results:
    context_piece = context_template.format(**doc)
    context_pieces.append(context_piece)

context = '\n\n'.join(context_piece)
prompt = prompt_template.format(question=query, context=context).strip()    

In [49]:
print('Q5 - What''s the length of the resulting prompt? (use the len function)')
print(len(prompt))

Q5 - Whats the length of the resulting prompt? (use the len function)
1514


In [57]:
encoding = tiktoken.encoding_for_model("gpt-4o")
tokens_in = len(encoding.encode(prompt))

print('Q6 - Use the encode function. How many tokens does our prompt have?')
print(tokens_in, 'toekns in')

Q6 - Use the encode function. How many tokens does our prompt have?
736 toekns in


In [58]:
answer = openai(prompt)
tokens_out=len(encoding.encode(answer))

print("BONUS 1- Let's send the prompt to OpenAI. What's the response?")
print(answer)
print(tokens_out, 'tokens out')

BONUS 1- Let's send the prompt to OpenAI. What's the response?
To execute a command in a running docker container, you can copy files from your local machine into the Docker container using the docker cp command. In the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:

COPY ["src/predict.py", "models/xgb_model.bin", "./"]
71 tokens out


In [59]:

# How much will it cost to run 1000 requests using the total tokens from previous answers?
fare_in=0.005
fare_out=0.015

#1000 requests
tokens_in= tokens_in * 1000
tokens_out= tokens_out * 1000

price_in = tokens_in*fare_in
price_out = tokens_out*fare_out

print('BONUS2 - How much will it cost to run 1000 requests using the total tokens from previous answers?\nInput: $0.005 / 1K tokens\nOutput: $0.015 / 1K tokens')
print('Total tkn entrada:',tokens_in,' = US$:', price_in)
print('Total tkn saida:',tokens_out,' = US$:', price_out)
print('Total US$:', price_out+price_in)

BONUS2 - How much will it cost to run 1000 requests using the total tokens from previous answers?
Input: $0.005 / 1K tokens
Output: $0.015 / 1K tokens
Total tkn entrada: 736000  = US$: 3680.0
Total tkn saida: 71000  = US$: 1065.0
Total US$: 4745.0
